In [1]:
from dataclasses import dataclass
from collections import defaultdict

In [2]:
data = open("input/13").read().splitlines()

In [3]:
turns = ["left", "straight", "right"]

In [4]:
@dataclass
class Cart:
    x: int
    y: int
    direction: str
    turn_idx: int = 0

In [5]:
grid = {}
carts = []
for r, row in enumerate(data):
    for c, elem in enumerate(row):
        pos = (r, c)
        if elem == " ":
            continue
        if elem == ">":
            grid[pos] = "-"
            carts.append(Cart(r, c, elem))
        elif elem == "<":
            grid[pos] = "-"
            carts.append(Cart(r, c, elem))
        elif elem == "v":
            grid[pos] = "|"
            carts.append(Cart(r, c, elem))
        elif elem == "^":
            grid[pos] = "|"
            carts.append(Cart(r, c, elem))
        else:
            grid[pos] = elem  

In [6]:
def sort_carts(carts):
    carts = sorted(carts, key=lambda c: (c.x, c.y))
    return carts

In [7]:
direction_map = {
    ">": (0, 1),
    "<": (0, -1),
    "v": (1, 0),
    "^": (-1, 0),
}

In [8]:
def set_cart_one(cart):
    d = {">": "^", "<": "v", "^": "<", "v": ">"}
    cart.direction = d[cart.direction]

def set_cart_two(cart):
    d = {">": "^", "<": "v", "^": ">", "v": "<"}
    cart.direction = d[cart.direction]

def set_cart_three(cart):
    d = {">": "v", "<": "^", "^": ">", "v": "<"}    
    cart.direction = d[cart.direction]

def set_cart_four(cart):
    d = {">": "v", "<": "^", "^": "<", "v": ">"}    
    cart.direction = d[cart.direction]

In [9]:
def tick(carts):
    carts = sort_carts(carts)
    crashed = set()
    for idx1, cart in enumerate(carts):
        if idx1 in crashed:
            continue
        pos = (cart.x, cart.y)
        cur_elem = grid[pos]
        xd, yd = direction_map[cart.direction]
        next_pos = (pos[0] + xd, pos[1] + yd)
        
        cart.x = next_pos[0]
        cart.y = next_pos[1]

        next_elem = grid[next_pos]
        if next_elem == "+":
            cur_turn = turns[cart.turn_idx % 3]
            cart.turn_idx += 1
            if cur_turn == "left":
                set_cart_one(cart)
            elif cur_turn == "right":
                set_cart_three(cart)
                
        elif next_elem == "/":
            set_cart_two(cart)

        elif next_elem == "\\":
            set_cart_four(cart)

        for idx2, other in enumerate(carts):
            if idx1 != idx2 and idx2 not in crashed:
                if cart.x == other.x and cart.y == other.y:
                    crashed.add(idx1)
                    crashed.add(idx2)
                    break
    return carts, crashed


In [10]:
part1 = None
while True:
    carts, crashed = tick(carts)
    if crashed and part1 is None:
        cart = carts[list(crashed)[0]]
        part1 = f"{cart.y},{cart.x}"
    carts = [cart for idx, cart in enumerate(carts) if idx not in crashed]
    if len(carts) == 1:
        break

In [11]:
print(f"Answer #1: {part1}")

Answer #1: 136,36


In [12]:
cart = carts[0]
part2 = f"{cart.y},{cart.x}"
print(f"Answer #2: {part2}")

Answer #2: 53,111


In [13]:
# Not used now but used it for debugging
def visualize():
    for r, row in enumerate(data):
        for c, elem in enumerate(row):
            p = tuple((r, c))
            if p in grid:
                cart_matching = []
                for cart in carts:
                    if cart.x == r and cart.y == c:
                        cart_matching.append(cart.direction)
                if len(cart_matching) == 1:
                    print(cart_matching[0], end="")
                elif len(cart_matching) > 1:
                    print("X", end="")
                else:
                    print(grid[p], end="")
            else:
                print(" ", end="")
        print()